In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/SST_2"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
bs=52 # 52 - Jeremey, 20 - default

In [5]:
bptt= 70 #70 - Jeremey, 35 - default

In [6]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [7]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
train_tokens.shape, valid_tokens.shape

((52, 30919), (52, 3417))

In [8]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [9]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [10]:
x.shape,y.shape

((52, 70), (52, 70))

In [11]:
x,y

(array([[   3,    4,    5,    2, ...,  285,   31,   99, 1242],
        [   3,    4,    5,    2, ...,    6, 7542,   19,   23],
        [   5,    2,    9,  261, ...,    5,    2,    8,   28],
        [3427,    8, 1543,   10, ...,  692,   11, 2006,   14],
        ...,
        [  13, 2068,    7,  352, ...,   66,    8,    7,  588],
        [   2,   16,  227,  166, ...,   16, 4688,  273, 7273],
        [  52,   22,  428,   43, ..., 1152,   18,   79, 2276],
        [1442,    6, 1150,   39, ...,   12,    3,    4,    5]]),
 array([[   4,    5,    2,  628, ...,   31,   99, 1242,   11],
        [   4,    5,    2,    7, ..., 7542,   19,   23, 7543],
        [   2,    9,  261,  172, ...,    2,    8,   28,   36],
        [   8, 1543,   10, 4003, ...,   11, 2006,   14,   19],
        ...,
        [2068,    7,  352,   11, ...,    8,    7,  588,  105],
        [  16,  227,  166, 1336, ..., 4688,  273, 7273,  293],
        [  22,  428,   43,   10, ...,   18,   79, 2276,   21],
        [   6, 1150,   39, 

In [12]:
x.shape,y.shape

((52, 70), (52, 70))

In [13]:
train_tokens.shape

(52, 30919)

## pre-trained AWD-LSTM

In [14]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [15]:
ls {PATH_AWD_LSTM}

bwd_wt103_enc.h5  fwd_wt103_enc.h5  itos_wt103.pkl  wt103_60002/
bwd_wt103.h5      fwd_wt103.h5      wt103_238642/   wt103_tiny.tgz


In [16]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [17]:
wgts

OrderedDict([('0.encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('0.encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                

In [18]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [19]:
wgts

OrderedDict([('encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                    

In [20]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [21]:
wgts['encoder.weight'].shape

torch.Size([238462, 400])

In [22]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [23]:
enc_wgts.shape, len(itos)

((238462, 400), 16206)

In [24]:
n_emb=enc_wgts.shape[1]

In [25]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [26]:
len(set(itos).difference(set(itos2)))

891

## Model Architecture

In [27]:
from torch import nn
import torch
from torch.autograd import Variable

In [28]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.5
wd=1e-7

In [29]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [30]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [31]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [32]:
new_w.shape

(16206, 400)

In [33]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([16206, 400]), torch.Size([16206, 400]))

In [34]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([16206, 400]), 400, 16206)

### Test if model forward works

In [35]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([16206, 400]), torch.Size([16206, 400]))

In [36]:
x.shape

(52, 70)

In [37]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [38]:

model=model.to(device)

## Learner

In [39]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1

    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.gen_hidden()
        #for k,i in enumerate(range(0,n_batch,self.bptt)):
        n_batch=iterator.shape[1]
        while i<n_batch-bptt:
            if mode_train:
                cust_bptt=self.bptt if np.random.random() < 0.95 else self.bptt//np.random.randint (2,4)
            else:
                cust_bptt=bptt
            seq_len=min(cust_bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
            k=k+1
            i=i+seq_len
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [40]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [41]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

442

In [42]:
model.freeze_embedding()

In [43]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [44]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [45]:
model.n_inp

16206

In [46]:
learner.run_epochs(train_tokens,valid_tokens,60)

Epoch:0 Loss:6.919100963676369
Epoch:0 Loss:6.919100963676369 Accuracy:0.13795176182443705 Loss:6.290080229441325 Accuracy:0.17168041101346412
Epoch:1 Loss:6.339776808517989
Epoch:1 Loss:6.339776808517989 Accuracy:0.16216519543307317 Loss:5.8777219752470655 Accuracy:0.1892227636029323
Epoch:2 Loss:5.991374429488024
Epoch:2 Loss:5.991374429488024 Accuracy:0.17336776694714628 Loss:5.520935734113057 Accuracy:0.2032566467920939
Epoch:3 Loss:5.691079924591874
Epoch:3 Loss:5.691079924591874 Accuracy:0.1832196998648939 Loss:5.2054474751154585 Accuracy:0.2169070631886522
Epoch:4 Loss:5.419516736715376
Epoch:4 Loss:5.419516736715376 Accuracy:0.1915554537187589 Loss:4.9079897701740265 Accuracy:0.2312729066858689
Epoch:5 Loss:5.168744438079767
Epoch:5 Loss:5.168744438079767 Accuracy:0.202082014955808 Loss:4.632564624150594 Accuracy:0.246348454306523
Epoch:6 Loss:4.941565928480173
Epoch:6 Loss:4.941565928480173 Accuracy:0.24297591590933862 Loss:4.386468718449275 Accuracy:0.3235748776545127
Epoch:7

Epoch:57 Loss:3.7589533546696536
Epoch:57 Loss:3.7589533546696536 Accuracy:0.31931260403083717 Loss:3.1159826467434564 Accuracy:0.43697346436480683
Epoch:58 Loss:3.755268177965231
Epoch:58 Loss:3.755268177965231 Accuracy:0.3212039959535264 Loss:3.1095045854647956 Accuracy:0.41088028252124786
Epoch:59 Loss:3.753373416892269
Epoch:59 Loss:3.753373416892269 Accuracy:0.3182781245232674 Loss:3.095067540804545 Accuracy:0.4119620124499003


In [47]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict')

In [48]:
model.unfreeze_embedding()

In [49]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [50]:
learner.run_epochs(train_tokens,valid_tokens,50)

Epoch:0 Loss:3.4230909852792095
Epoch:0 Loss:3.4230909852792095 Accuracy:0.4609213714173298 Loss:2.618056063850721 Accuracy:0.5857944451272488
Epoch:1 Loss:3.2049191406794955
Epoch:1 Loss:3.2049191406794955 Accuracy:0.4862217019547473 Loss:2.478322913249334 Accuracy:0.5971211368838946
Epoch:2 Loss:3.074958617952135
Epoch:2 Loss:3.074958617952135 Accuracy:0.49774672004911635 Loss:2.359963600834211 Accuracy:0.6113667823374271
Epoch:3 Loss:2.970897956470867
Epoch:3 Loss:2.970897956470867 Accuracy:0.5067141367183937 Loss:2.2538239657878876 Accuracy:0.622075350334247
Epoch:4 Loss:2.875206515424267
Epoch:4 Loss:2.875206515424267 Accuracy:0.515497596723806 Loss:2.157458650569121 Accuracy:0.6333104732135931
Epoch:5 Loss:2.7953958213329315
Epoch:5 Loss:2.7953958213329315 Accuracy:0.5232134187561378 Loss:2.0832270880540213 Accuracy:0.6418498506148657
Epoch:6 Loss:2.7196535341507566
Epoch:6 Loss:2.7196535341507566 Accuracy:0.5302085997124689 Loss:2.009914495050907 Accuracy:0.6528388584653536
Epoc

In [51]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict_unfreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict_unfreeze')

In [52]:
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type language_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [53]:
learner.run_epochs(train_tokens,valid_tokens,20)

Epoch:0 Loss:1.8093397842658745
Epoch:0 Loss:1.8093397842658745 Accuracy:0.6525833670909588 Loss:1.2510008563597996 Accuracy:0.7667811674376329
Epoch:1 Loss:1.80106171261659
Epoch:1 Loss:1.80106171261659 Accuracy:0.6536564950648259 Loss:1.2444622615973155 Accuracy:0.7679430283606052
Epoch:2 Loss:1.795681797116008
Epoch:2 Loss:1.795681797116008 Accuracy:0.654837611626842 Loss:1.241359956562519 Accuracy:0.7683780118823051
Epoch:3 Loss:1.7904357753986875
Epoch:3 Loss:1.7904357753986875 Accuracy:0.6554506166012527 Loss:1.2385850995779037 Accuracy:0.7693624372283617
Epoch:4 Loss:1.7820135477398122
Epoch:4 Loss:1.7820135477398122 Accuracy:0.6568631501868367 Loss:1.234282764295737 Accuracy:0.7701808934410413
Epoch:5 Loss:1.77428889936871
Epoch:5 Loss:1.77428889936871 Accuracy:0.658147214518653 Loss:1.2304666936397552 Accuracy:0.7709707332154115
Epoch:6 Loss:1.7712087833855064
Epoch:6 Loss:1.7712087833855064 Accuracy:0.6585732303707805 Loss:1.225143591562907 Accuracy:0.7718063505987326
Epoch:7

In [54]:
learner.run_epochs(train_tokens,valid_tokens,20)

Epoch:0 Loss:1.7090238940533193
Epoch:0 Loss:1.7090238940533193 Accuracy:0.6688894502965314 Loss:1.1833711539705594 Accuracy:0.7791037497421106
Epoch:1 Loss:1.704445949826625
Epoch:1 Loss:1.704445949826625 Accuracy:0.6700475326290837 Loss:1.181585930287838 Accuracy:0.7795272792379061
Epoch:2 Loss:1.6992818966376042
Epoch:2 Loss:1.6992818966376042 Accuracy:0.6708904078840154 Loss:1.1794891407092412 Accuracy:0.779527281721433
Epoch:3 Loss:1.6943875393678438
Epoch:3 Loss:1.6943875393678438 Accuracy:0.6719206421910928 Loss:1.176107866068681 Accuracy:0.780551773806413
Epoch:4 Loss:1.6918254281980876
Epoch:4 Loss:1.6918254281980876 Accuracy:0.6722753766087184 Loss:1.1728282868862152 Accuracy:0.780671967814366
Epoch:5 Loss:1.6892850482856834
Epoch:5 Loss:1.6892850482856834 Accuracy:0.6726599445709815 Loss:1.1702159841855366 Accuracy:0.7813187179466089
Epoch:6 Loss:1.684846241564021
Epoch:6 Loss:1.684846241564021 Accuracy:0.6731853541936684 Loss:1.1684076587359111 Accuracy:0.7818738954762617
E

In [55]:
learner.run_epochs(train_tokens,valid_tokens,30)

Epoch:0 Loss:1.6414882575764376
Epoch:0 Loss:1.6414882575764376 Accuracy:0.6812373098464832 Loss:1.1441115277508895 Accuracy:0.7863324582576752
Epoch:1 Loss:1.6387577438563632
Epoch:1 Loss:1.6387577438563632 Accuracy:0.6811340147988838 Loss:1.143005261818568 Accuracy:0.7856914301713308
Epoch:2 Loss:1.6349285091672625
Epoch:2 Loss:1.6349285091672625 Accuracy:0.6819944330624171 Loss:1.1388371425370376 Accuracy:0.7866815862556299
Epoch:3 Loss:1.631730903326544
Epoch:3 Loss:1.631730903326544 Accuracy:0.6825169131455832 Loss:1.1391915467878182 Accuracy:0.7871222893397013
Epoch:4 Loss:1.6304739414996312
Epoch:4 Loss:1.6304739414996312 Accuracy:0.6827009507254058 Loss:1.1370935700833797 Accuracy:0.787282545119524
Epoch:5 Loss:1.6270468792735606
Epoch:5 Loss:1.6270468792735606 Accuracy:0.6833858114652782 Loss:1.1348146225015323 Accuracy:0.7875973408420881
Epoch:6 Loss:1.6226344021161396
Epoch:6 Loss:1.6226344021161396 Accuracy:0.6841492301887936 Loss:1.1341620857516925 Accuracy:0.7877575928966

In [56]:
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm_final')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm_final')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm_final')

In [57]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','wb'))

In [3]:
import numpy as np

In [4]:
np.exp(1.10)

3.0041660239464334